<a href="https://colab.research.google.com/github/talhaanwarch/youtube-tutorials/blob/main/eeg_epilepsy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unzip the data

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#unzip the files
from zipfile import ZipFile
filepath = r"D:\6th Sem Docs\project\ptsd detection\epilepsy-detection-using-ML\EEGs_Guinea-Bissau.zip"
data = ZipFile(filepath)
data.extractall()

# Read data

In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [8]:
meta_df=pd.read_csv('https://zenodo.org/record/1252141/files/metadata_guineabissau.csv')
meta_df.head()

,subject.id,Group,Eyes.condition,Remarks,recordedPeriod,startTime
0,1,Epilepsy,closed-3min-then-open-2min,by 45s reposition electrodes,301,27/5/2020 14:33
1,2,Control,open-3min-then-closed-2min,NaN,309,26/5/2020 22:44
2,3,Epilepsy,closed-3min-then-open-2min,NaN,309,27/5/2020 14:26
3,4,Epilepsy,closed-3min-then-open-2min,"Green lights not shown, but good EEG traces",299,27/5/2020 15:23
4,5,Control,closed-3min-then-open-2min,NaN,302,23/5/2020 19:09


In [9]:
print(meta_df.shape)

(97, 6)


In [10]:
#now i need to seprate Epilepsy vs Control subjects
EP_sub=meta_df['subject.id'][meta_df['Group']=='Epilepsy']
CT_sub=meta_df['subject.id'][meta_df['Group']=='Control']

In [11]:
#read csv files
Epilepsy=[pd.read_csv('EEGs_Guinea-Bissau/signal-{}.csv.gz'.format(i), compression='gzip') for i in  EP_sub]
Control=[pd.read_csv('EEGs_Guinea-Bissau/signal-{}.csv.gz'.format(i), compression='gzip') for i in  CT_sub]

In [12]:
print(Epilepsy[0].shape)
print(Control[0].shape)

(38528, 36)
(39552, 36)


In [13]:
Epilepsy[0].head()

,Unnamed: 0,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,...,CQ_F3,CQ_P7,CQ_P8,CQ_F4,CQ_AF3,CQ_FC5,CQ_O1,CQ_T8,CQ_F8,CQ_DRL
0,1,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,...,4,4,4,4,4,4,4,4,4,4
1,2,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,...,4,4,4,4,4,4,4,4,4,4
2,3,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,...,4,4,4,4,4,4,4,4,4,4
3,4,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,...,4,4,4,4,4,4,4,4,4,4
4,5,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,...,4,4,4,4,4,4,4,4,4,4


In [14]:
#remove non eeg channels
Epilepsy=[i.iloc[:,1:15] for i in  Epilepsy]
Control=[i.iloc[:,1:15] for i in  Control]

In [15]:
Epilepsy[0].head()

,AF3,AF4,F3,F4,F7,F8,FC5,FC6,O1,O2,P7,P8,T7,T8
0,4426.153846,3994.871795,4408.205128,3847.692308,4690.256410,3895.897436,4702.051282,3914.871795,4049.743590,4297.948718,4186.153846,4132.820513,4106.666667,4076.410256
1,4420.512821,3986.666667,4394.358974,3836.923077,4678.461538,3886.666667,4696.410256,3910.769231,4054.358974,4295.897436,4187.179487,4126.666667,4105.641026,4065.128205
2,4413.846154,3986.153846,4386.666667,3831.794872,4654.871795,3881.025641,4690.769231,3908.205128,4066.666667,4301.025641,4188.205128,4123.076923,4103.076923,4063.589744
3,4407.692308,3984.615385,4384.102564,3832.820513,4644.615385,3883.076923,4686.153846,3910.256410,4063.076923,4298.974359,4184.615385,4127.179487,4095.384615,4071.282051
4,4407.179487,3978.974359,4382.564103,3832.307692,4647.692308,3878.974359,4685.641026,3903.076923,4057.948718,4295.384615,4182.564103,4128.205128,4090.769231,4064.102564


# Convert to MNE object

In [16]:
import mne
def convertDF2MNE(sub):
    info = mne.create_info(list(sub.columns), ch_types=['eeg'] * len(sub.columns), sfreq=128)
    info.set_montage('standard_1020')
    data=mne.io.RawArray(sub.T, info)
    data.set_eeg_reference()
    data.filter(l_freq=0.1,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    epochs=epochs.drop_bad()

    return epochs

In [17]:
%%capture
#Convert each dataframe to mne object
Epilepsy=[convertDF2MNE(i) for i in  Epilepsy]
Control=[convertDF2MNE(i) for i in  Control]

In [18]:
%%capture
#concatenate the epochs
Epilepsy_epochs=mne.concatenate_epochs(Epilepsy)
Control_epochs=mne.concatenate_epochs(Control)


# Create labels and groups

In [19]:
Epilepsy_group=np.concatenate([[i]*len(Epilepsy[i]) for i in range(len(Epilepsy))])#create a list of list where each sub list corresponds to subject_no
Control_group=np.concatenate([[i]*len(Control[i]) for i in range(len(Control))])#create a list of list where each sub list corresponds to subject_no

Epilepsy_label=np.concatenate([[0]*len(Epilepsy[i]) for i in range(len(Epilepsy))])
Control_label=np.concatenate([[1]*len(Control[i]) for i in range(len(Control))])

In [20]:
Epilepsy_group.shape,Control_group.shape,Epilepsy_label.shape,Control_label.shape

((3995,), (3461,), (3995,), (3461,))

In [21]:
#combine data
data=mne.concatenate_epochs([Epilepsy_epochs,Control_epochs])
group=np.concatenate((Epilepsy_group,Control_group))
label=np.concatenate((Epilepsy_label,Control_label))
print(len(data),len(group),len(label))

Not setting metadata
7456 matching events found
No baseline correction applied
7456 7456 7456


# Feature Extraction - Power spectral density
The power spectral density of a signal is a measure of how much power the signal has at each different frequency. The power spectral density (power spectrum) reflects the ‘frequency content’ of the signal or the distribution of signal power over frequency.  


In [22]:
# source: https://mne.tools/stable/auto_tutorials/clinical/60_sleep.html#sphx-glr-auto-tutorials-clinical-60-sleep-py
# from mne.time_frequency import psd_welch
def eeg_power_band(epochs):
    """EEG relative power band feature extraction.

    This function takes an ``mne.Epochs`` object and creates EEG features based
    on relative power in specific frequency bands that are compatible with
    scikit-learn.

    Parameters
    ----------
    epochs : mne.Epochs
        The data.

    Returns
    -------
    X : numpy array of shape [n_samples, n_bands * n_channels]
        Transformed data.
    """
    # Define specific frequency bands
    FREQ_BANDS = {
        "delta": [0.5, 4.5],
        "theta": [4.5, 8.5],
        "alpha": [8.5, 11.5],
        "sigma": [11.5, 15.5],
        "beta": [15.5, 30],
        "gamma": [30, 45],
    }

    # Compute the PSD using the compute_psd method
    psd = epochs.compute_psd(method="welch", fmin=0.5, fmax=45, picks="eeg")
    psds, freqs = psd.get_data(return_freqs=True)  # Get PSD values and frequencies

    # Normalize the PSDs
    psds /= np.sum(psds, axis=-1, keepdims=True)

    # Extract mean PSD for each frequency band
    X = []  # Initialize feature list
    for fmin, fmax in FREQ_BANDS.values():
        # Identify frequencies within the band
        freq_mask = (freqs >= fmin) & (freqs < fmax)
        # Compute mean PSD for the band and add to the feature list
        psds_band = psds[:, :, freq_mask].mean(axis=-1)
        X.append(psds_band)

    # Concatenate the mean PSDs for all bands
    return np.concatenate(X, axis=1)#Concatenate the mean PSDs for each band into a single feature vector

# Classification 5-fold

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [24]:
print(mne.__version__)

1.8.0


In [25]:
%%capture
features=[]
for d in range(len(data)):#get features from each epoch and save in a list
  features.append(eeg_power_band(data[d]))

In [45]:
#convert list to array
features=np.concatenate(features)
features.shape

(626304,)

In [27]:
#do 5 fold cross validation
clf=RandomForestClassifier()
rfc_accuracies=cross_val_score(clf, features,label,groups=group,cv=5)
print('Five fold accuracies',accuracies)
print('Average accuracy',np.mean(accuracies))

NameError: name 'accuracies' is not defined

In [ ]:
#do 5 fold cross validation
from xgboost import XGBClassifier
xgbC=XGBClassifier(max_depth = 5, n_estimator = 200, use_label_encoder = False)
xgb_accuracies=cross_val_score(xgbC, features,label,groups=group,cv=5)
print('Five fold accuracies',xgb_accuracies)
print('Average accuracy',np.mean(xgb_accuracies))

Five fold accuracies [0.72386059 0.71294433 0.64788732 0.75519785 0.72501677]
Average accuracy 0.7129813734956657


In [25]:
xgbC.fit(features, label)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimator=200, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [54]:
from ml_models import dl_model

ImportError: cannot import name 'dl_model' from 'ml_models' (unknown location)

In [48]:
dl_model = dl_model(features, label)

history = dl_model.fit(
    features,  # Your input features (numpy.ndarray)
    label,    # Corresponding labels
    batch_size=32,   # Adjust based on dataset size
    epochs=20,       # Adjust based on convergence
    validation_split=0.2,  # Reserve 20% of data for validation
    shuffle=True
)

NameError: name 'dl_model' is not defined

In [27]:
import pickle

with open(r"D:\6th Sem Docs\project\ptsd detection\epilepsy detector\models\xgboost_model.pkl", "wb") as file:
    pickle.dump(xgbC, file)

print("Model saved as xgboost_model.pkl")

Model saved as xgboost_model.pkl
